In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("/home/iftekhar/AI-system/retrieval_Model/Page_Ranking_Experiment/experiment_amplified_dataset.csv")
df = df.iloc[:,1:]
df = df.rename(columns={"Text":"Data"})
df.PageID = df.PageID.apply(lambda x: str(x).replace("Page_",""))
df.head()

,PageID,Data
0,228,実行 管理 オフライン 端末 MobiControl
1,28,Active Service Domain Azure Directory
2,409,社外 なく 登録 防ぐ セキュリティ
3,396,名前 プロファイル ペン 構成 ドラフト
4,257,WiFi 不 電話 回線 圏外


In [2]:
from make_question import make_question_from_sentence


In [3]:
question_saver = []
PageID_saver = []

count = 0
for index, col in df.iterrows():
    question_saver.append(make_question_from_sentence(col["Data"]))
    PageID_saver.append(col["PageID"])
    if count > 1000:
        break
    count += 1

In [4]:
questions_samples = pd.DataFrame(zip(question_saver, PageID_saver), columns=['Question', 'PageID'])
questions_samples.head()


,Question,PageID
0,実行 管理 オフライン MobiControl 何ですか,228
1,Active Service Domain Directory どちら,28
2,社外 登録 防ぐ セキュリティ 作りたい,409
3,プロファイル ペン 構成 ドラフト 何ですか,396
4,WiFi 電話 回線 圏外 ついて教えてください,257


In [9]:
len(questions_samples)

1002

In [6]:
import fasttext
fast_model = fasttext.load_model("page_ranker.bin")

In [5]:
from make_question import make_question_from_sentence

In [6]:
query = make_question_from_sentence("という 環境 変数 値 DoCoMo ない なら ば : next")

In [7]:
def rank_result_collection(query, fast_model):
    result = fast_model.predict(query,k=3)
    ID_collect = []
    score_collect = []
    for items in result:
        for item in items:
            if str(item).find("__label") != -1:
                ID_collect.append(str(item).replace("__label__Page_",""))
            else:
                score_collect.append(item)
                
    return zip(ID_collect, score_collect) 

In [50]:
len(questions_corpus)

1158258

In [12]:
from ranking_help_methods import mean_reciprocal_rank_score

In [13]:
def get_score_details_record(df, questions_samples, fast_model):
    sample_count = 0
    sum_score = 0
    container = []

    for index, col in questions_samples.iterrows():
        query = str(col['Question'])
        #print(query)
        rank_list = rank_result_collection(query, fast_model)
        
        page_answers = []
        prediction_scores = []
        for ids, score in rank_list:
            #print(ids, score, col["PageID"])
            page_answers.append(ids)
            prediction_scores.append(score)
        
        MRR_score = mean_reciprocal_rank_score(col['PageID'], page_answers)
        sum_score += MRR_score

        container.append([MRR_score, col['PageID'], page_answers, prediction_scores, col['Question']])
        sample_count += 1
        
    result = pd.DataFrame(container, columns=['score', 'actual_answer', 'page_answers', 'prediction_scores',
                                              'query'])
    result.to_csv('performance.csv')
    score = sum_score / sample_count

    return score


ranks = get_score_details_record(df, questions_samples, fast_model)
ranks

0.29750499001996017

NameError: name 'result' is not defined